Import du modèle

In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image


In [2]:
np.random.seed(3)

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask 
        # if borders:
        #     import cv2
        #     contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        #     # Try to smooth contours
        #     contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        #     cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1) 

    ax.imshow(img)

In [34]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_large.pt"
checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_tiny.pt"
# model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
# predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))

sam2 = build_sam2(model_cfg, checkpoint, device="cuda", apply_postprocessing=False)

mask_generator = SAM2AutomaticMaskGenerator(sam2)
# with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    # predictor.set_image("/home/killian/sam2")
    # masks, _, _ = predictor.predict("0")
    # 🔹 Chemins vers le modèle et la configuration
#checkpoint = "./checkpoints/sam2_hiera_large.pt"
#model_cfg = "sam2_hiera_l.yaml"


Hyperparamètres à régler

DataLoader

In [ ]:
from torch.utils.data import Dataset
import glob
import os  # Nécessaire pour os.path.join
import numpy as np  # Pour convertir en numpy array
from PIL import Image  # Pour ouvrir les images

# on définit le datatloader
class DataLoaderSegmentation(Dataset):

    def __init__(self, folder_path, transforms=None):
        super(DataLoaderSegmentation, self).__init__()

        self.img_files = glob.glob(os.path.join(folder_path, '*.jpg'))

        # Print des fichiers pour débogage
        # print(self.img_files)
        self.mask_files = []

        for img_path in self.img_files:
            # Créer le chemin pour le masque correspondant
            self.mask_files.append(os.path.join(folder_path, f'{img_path[:-4]}_mask.png'))

        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_files[index]
        mask_path = self.mask_files[index]

        # Ouvrir l'image et la convertir en RGB
        image = Image.open(img_path)
        image = np.array(image.convert("RGB"))

        # Ouvrir le masque et le convertir en RGB
        label = Image.open(mask_path)
        label = np.array(label.convert("RGB"))

        # Si des transformations sont spécifiées, les appliquer (non activées ici)
        if self.transforms:
            #  image = self.transforms(torch.from_numpy(image).float())
              image = self.transforms(torch.as_tensor(np.array(image).astype('float')))


        # Retourner l'image et le masque en tant que tenseurs PyTorch
        return image, torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)


permet de georéférencé nos images

In [5]:
import os
import tempfile

import kornia.augmentation as K
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as T

#from torchgeo.datasets import EuroSAT100
from torchgeo.datasets import RasterDataset, stack_samples
from torchgeo.samplers import GridGeoSampler, RandomGeoSampler
from torchgeo.models import ResNet18_Weights, resnet18
from torchgeo.transforms import AugmentationSequential

import matplotlib.pyplot as plt
torch.manual_seed(0)

/home/killian/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import sys
import os
import rasterio
from rasterio.transform import from_origin

def georeference_tiff(input_filename, output_filename):
    # Open the input raster file
    with rasterio.open(input_filename) as src:
        # Get raster dimensions
        width = src.width
        height = src.height

        # Set an arbitrary transformation
        transform = from_origin(0, 0, 1, 1)  # Adjust the pixel size as needed

        # Create the output raster profile with an arbitrary CRS (coordinate reference system)
        profile = src.profile
        profile.update(transform=transform, crs='EPSG:4326')  # EPSG:4326 is WGS 84

        # Modify output filename with "geo_" prefix
        output_filename = output_filename.replace(os.path.basename(output_filename), "geo_" + os.path.basename(output_filename))

        # Write to the output raster file
        with rasterio.open(output_filename, 'w', **profile) as dst:
            data = src.read()
            dst.write(data)

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python script.py input_file1.tif input_file2.tif ...")
        sys.exit(1)

    input_files = sys.argv[1:]
    for input_file in input_files:
        output_file = "geo_" + os.path.basename(input_file)
        
        georeference_tiff('/home/killian/data2025/15485/X200_15485_PB1.tif', '/home/killian/data2025/15485/X200_15485_PB1.tif')
        print(f"Georeferenced TIFF saved as {output_file}")
 

/home/killian/.local/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/killian/.local/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Georeferenced TIFF saved as geo_kernel-v3f11e16ea8e2f7ea713cb7064461c49ae2c3a1199.json


Dataset

In [38]:

import os
import torchvision.transforms as T
from torch.utils.data import DataLoader

MEANS = [122.39, 118.23, 98.1]
SDS = [39.81, 37.33, 33.04]

test_transform = T.Compose([T.ConvertImageDtype(torch.float32),
                            T.Normalize(MEANS,SDS),])


dataset = DataLoaderSegmentation('/home/killian/Annotations/Annotations', transforms=test_transform)
# dataset = DataLoaderSegmentation('/home/killian/Annotations/Annotations')
dataloader = DataLoader(dataset=dataset, batch_size=1)

In [36]:
#Hyperparametres
model = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=20,  # Plus de points pour capturer les details
    points_per_batch=30,  # Augmenter pour calculer le nbre de points pris en meme temps (/!\ GPU)
    pred_iou_thresh=0.7,  # Reduire pour accepter plus de mask
    stability_score_thresh=0.80,  # Rzduire pour ne pas exclure trop de mask
    stability_score_offset=0.8,
    crop_n_layers=2,  # Ammeliore la segmentation des petites structures
    box_nms_thresh=0.80,  # Eviter la suppression excessive de petite structure
    crop_n_points_downscale_factor=2,  # Adapter aux images a haute resolution
    min_mask_region_area=15.0,  # Conserver plus de petits objets
    use_m2m=True,  # Mode avancé 
)

In [40]:
#on definit une fonction pour concatener tout nos mask 
def merge_preds(pred):
    res = torch.stack([torch.Tensor(mask['segmentation']) for mask in pred])
    res.shape
    res = res.any(dim=0)
    return res

Modele run:

In [2]:
import os
import csv
import torch
import torchmetrics
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Définition du device
device = "cuda"

# Définition des métriques
miou = torchmetrics.JaccardIndex(task='binary', num_classes=1).to(device)
precision = torchmetrics.Precision(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
f1_score = torchmetrics.F1Score(task='binary').to(device)

# Dossier de sortie
output_dir = "/home/killian/sam2/predictions"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV pour les métriques globales
csv_file = os.path.join(output_dir, "sam2_results.csv")
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "IoU", "Precision", "Recall", "F1_Score", "Num_Masks", "Num_Masks_Label"])

# Fichier CSV pour les mesures des masques
csv_masks_file = os.path.join(output_dir, "mask_measurements.csv")
with open(csv_masks_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "Mask_ID", "Centroid_X", "Centroid_Y", "Area", "Equivalent_Diameter"])

size_threshold = 10000  # Seuil pour filtrer les objets trop grands

# Variables pour les moyennes
total_miou = total_precision = total_recall = total_f1 = total_images = 0

for i, (batch, mask) in enumerate(dataloader):
        print(f"Image {i+1}/{len(dataloader)} - Avancement : {i/len(dataloader):.2%}", end='\r')
        batch = batch.to(device)

        for idx, (image, label) in enumerate(zip(batch, mask)):

            image_np = image.cpu().numpy()
            pred = model.generate(image_np)
            
            res_tensor = torch.stack([torch.tensor(m['segmentation'], dtype=torch.bool) for m in pred])
            filtered_tensor = res_tensor[res_tensor.sum(dim=(1, 2)) <= size_threshold]
            res_merge = filtered_tensor.any(dim=0).to(device)
            
            label_np = np.array(label)
            trach = torch.as_tensor(label_np[:, :, 2], dtype=torch.bool, device=device) if label_np.ndim == 3 and label_np.shape[-1] >= 3 else torch.as_tensor(label_np, dtype=torch.bool, device=device)
            
            num_masks_label = (label_np > 0).sum()
            
            # Calcul des métriques
            img_miou = miou(res_merge, trach).item()
            img_precision = precision(res_merge, trach).item()
            img_recall = recall(res_merge, trach).item()
            img_f1 = f1_score(res_merge, trach).item()
            
            total_miou += img_miou
            total_precision += img_precision
            total_recall += img_recall
            total_f1 += img_f1
            total_images += 1
            
            # Sauvegarde des résultdataloader
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([f"Image_{i}_{idx}", img_miou, img_precision, img_recall, img_f1, filtered_tensor.shape[0], num_masks_label])
            
            # Mesure des aires et centroïdes des masques
            for mask_id, mask_pred in enumerate(filtered_tensor):
                mask_np = mask_pred.cpu().numpy().astype(np.uint8)
                num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_np, connectivity=8)
                
                with open(csv_masks_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    for j in range(1, num_labels):
                        x, y = centroids[j]
                        area = stats[j, cv2.CC_STAT_AREA]
                        equivalent_diameter = np.sqrt(4 * area / np.pi)
                        writer.writerow([f"Image_{i}_{idx}", mask_id, x, y, area, equivalent_diameter])
            
            # Sauvegarde des images
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.title("Prédiction")
            plt.imshow(res_merge.cpu().numpy(), cmap='gray')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.title("Label")
            plt.imshow(trach.cpu().numpy(), cmap='gray')
            plt.axis('off')
            
            plt.savefig(os.path.join(output_dir, f"Image_{i}_{idx}.png"))
            plt.close()

if total_images > 0:
    mean_miou = total_miou / total_images
    mean_precision = total_precision / total_images
    mean_recall = total_recall / total_images
    mean_f1 = total_f1 / total_images
    
    print(f"mIoU moyen: {mean_miou:.4f}")
    print(f"Précision moyenne: {mean_precision:.4f}")
    print(f"Rappel moyen: {mean_recall:.4f}")
    print(f"F1-Score moyen: {mean_f1:.4f}")


NameError: name 'dataloader' is not defined

Modèle de seullage avec Opencv

In [78]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchmetrics
import cv2

# Définition du device
device = "cuda"

# Définition des métriques
miou = torchmetrics.JaccardIndex(task='binary', num_classes=1).to(device)
precision = torchmetrics.Precision(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
f1_score = torchmetrics.F1Score(task='binary').to(device)

# Dossier de sortie
output_dir = "/home/killian/Annotations/Seuillage"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV pour les métriques globales
csv_file = os.path.join(output_dir, "seuillage_results.csv")
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "IoU", "Precision", "Recall", "F1_Score", "Num_Masks", "Num_Masks_Label"])

# Fichier CSV pour les mesures des masques
csv_masks_file = os.path.join(output_dir, "mask_measurements.csv")
with open(csv_masks_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "Mask_ID", "Centroid_X", "Centroid_Y", "Area", "Equivalent_Diameter"])

# Variables pour les moyennes
total_miou = total_precision = total_recall = total_f1 = total_images = 0

for idx, (image, mask) in enumerate(dataloader):
    print(f"{idx}/{len(dataloader)}", end='\r')
    img_name = dataset.img_files[idx].split("/")[-1]
    
    image = image.squeeze(0).cpu().numpy()
    
    mask = mask.squeeze(0).cpu().numpy()
    mask = mask[:,:,2] # keep only tracheid labels
    
    gray = np.mean(image, axis=2) if image.ndim == 3 else image
    _, binary = cv2.threshold(gray.astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    processed = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(processed, connectivity=8)
    
    mask_bin = (mask > 0).astype(np.uint8)
    
    processed_bin = (processed > 0).astype(np.uint8)
    
    res_merge = torch.tensor(processed_bin, dtype=torch.bool, device=device)
    
    trach = torch.tensor(mask_bin, dtype=torch.bool, device=device)
    
    img_miou = miou(res_merge, trach).item()
    img_precision = precision(res_merge, trach).item()
    img_recall = recall(res_merge, trach).item()
    img_f1 = f1_score(res_merge, trach).item()
    
    total_miou += img_miou
    total_precision += img_precision
    total_recall += img_recall
    total_f1 += img_f1
    total_images += 1
    
    num_masks_label = (mask_bin > 0).sum()
    
    # Sauvegarde des mesures des 3, 640, 640)masques
    with open(csv_masks_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        for i in range(1, num_labels):
            x, y = centroids[i]
            area = stats[i, cv2.CC_STAT_AREA]
            equivalent_diameter = np.sqrt(4 * area / np.pi)
            writer.writerow([img_name, i, x, y, area, equivalent_diameter])
    
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Image d'origine")
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title("Prédiction")
    pred_image = res_merge.cpu().numpy()
    for centroid in centroids:
        plt.scatter(centroid[0], centroid[1], color='red', s=50, marker='x')
    plt.imshow(pred_image, cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.title("Label")
    plt.imshow(trach.cpu().numpy(), cmap='gray')
    plt.axis('off')
    
    plt.savefig(os.path.join(output_dir, f"Image_{idx}.png"))
    plt.close()
    
if total_images > 0:
    print(f"mIoU moyen: {total_miou / total_images:.4f}")
    print(f"Précision moyenne: {total_precision / total_images:.4f}")
    print(f"Rappel moyen: {total_recall / total_images:.4f}")
    print(f"F1-Score moyen: {total_f1 / total_images:.4f}")


ValueError: not enough values to unpack (expected 2, got 1)

In [32]:
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS>=178956970

extraire_bandes_images('/home/killian/data2025/11478')
extraire_bandes_images('/home/killian/data2025/13823')
extraire_bandes_images('/home/killian/data2025/15485')
extraire_bandes_images('/home/killian/data2025/15492')
extraire_bandes_images('/home/killian/data2025/TGV4')
extraire_bandes_images('/home/killian/data2025/TGV5')

Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/1147

In [38]:
import os
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as T
# import rasterio
import matplotlib.pyplot as plt
import tifffile

class CustomDataset(Dataset):

    def __init__(self,input_raster, tile_size=640, stride=10,transforms=None):

        # raster = rasterio.open(input_raster).read()
        # raster = plt.imread(input_raster)
        raster = tifffile.imread(input_raster)
        
        self.raster = raster[:,:tile_size,:]
        print(self.raster.shape)
        self.tile_size = tile_size
        self.stride = stride
        self.transforms = transforms

    def __len__(self):
        return int(self.raster.shape[0]/self.tile_size)
        

    def __getitem__(self, idx):
        
        if idx == 0:
            start_y = 0
            end_y = self.tile_size
        
        else:
            start_y = idx*self.tile_size - self.stride
            end_y = (idx+1)*self.tile_size - self.stride

        data = self.raster[start_y:end_y,:,:]
        data = data.swapaxes(0,2)
        # data = (data - np.min(data)) / (np.max(data) - np.min(data))
        # if self.transforms:
        #     data = self.transforms(data)
        return data
        
 

In [39]:
from torch.utils.data import DataLoader

dataset = CustomDataset('/home/killian/data2025/15485/X200_15485_PB1.tif', tile_size=640, stride=10)
dataloader = DataLoader(dataset,batch_size=1, shuffle=False)


(8935, 640, 3)


In [40]:
len(dataloader)

13

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

# Dossier de sortie
output_dir = "/home/killian/sam2/inferences"
os.makedirs(output_dir, exist_ok=True)
print('plop')

# Boucle d'inférence
for i, image in enumerate(dataloader):
    print(f"Image {i+1}/{len(dataloader)} - Avancement : {i/len(dataloader):.2%}", end='\r')

   
    # image = torch.tensor(image, dtype=torch.float32).to(device)
    
    # Inférence avec le modèle
    # image_np = image.squeeze(0).swapaxes(0, 2).cpu().numpy()
    image_np = image.squeeze(0).cpu().numpy()
    # print(image_np.shape)
    # print(image_np.max())
    
    image_np = image_np.swapaxes(0,2)
    # image_np = np.asarray(image_np.astype(np.float32))
    # print(type(image_np))
    # image_pil = Image.fromarray(image_np)
    # print(type(image_np))
    pred = model.generate(image_np)
    # Fusion des masques (si applicable)
    res_tensor = torch.stack([torch.tensor(m['segmentation'], dtype=torch.bool) for m in pred])
    res_merge = res_tensor.any(dim=0)
    
    # Sauvegarde des images
    plt.figure(figsize=(6, 6))
    plt.imshow(res_merge.cpu().numpy(), cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, f"Image_{i}.png"))
    plt.close()

plop
(3, 640, 640)Avancement : 0.00%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 7.69%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 15.38%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 23.08%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 30.77%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 38.46%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 46.15%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 53.85%
255
<class 'numpy.ndarray'>
(3, 640, 640)Avancement : 61.54%
255
<class 'numpy.ndarray'>
(3, 640, 640) Avancement : 69.23%
255
<class 'numpy.ndarray'>
(3, 640, 640) Avancement : 76.92%
255
<class 'numpy.ndarray'>
(3, 640, 640) Avancement : 84.62%
255
<class 'numpy.ndarray'>
(3, 640, 640) Avancement : 92.31%
255
<class 'numpy.ndarray'>
